In [1]:
from _settings import jupyer_notebook_print_many
from _visualizer import *
from _utils_torch import *

import cv2
import numpy as np
import matplotlib.pyplot as plt

from torch import nn
from _layers import LinearBlock, Conv2dBlock, ResBlock
from utils import *
from networks import *

jupyer_notebook_print_many()
%matplotlib inline

### Set up

In [2]:
raw = plt.imread('./Input/Images/birds.png')[:, :, :3]

config = get_config('./config/random_sample.yaml')
config = adjust_scale_factor_by_image(raw, config)

In [3]:
scale_factor = config['scale_factor']
num_scale = config['num_scale']

In [4]:
width, height, _ = raw.shape

height_pyramid, width_pyramid, real_pyramid = [], [], []
for i in range(num_scale, -1, -1):
    multiplier = (1 / scale_factor) ** i
    
    height_scaled = int(round(height * multiplier))
    width_scaled = int(round(width * multiplier))
    
    height_pyramid.append(height_scaled)
    width_pyramid.append(width_scaled)
    
    raw_scaled = cv2.resize(raw, (height_scaled, width_scaled))
    raw_scaled = torch.tensor(np.transpose(raw_scaled, [2, 0, 1])[np.newaxis])
    real_pyramid.append(raw_scaled)
    
generator_pyramid, discriminator_pyramid = [], []
for scale in range(config['num_scale']):
    dim = 32 * 2 ** (scale // 4)      # increase dim by a factor of 2 every 4 scales
    generator_pyramid.append(Generator(input_dim=3, dim=dim, n_layers=5))
    discriminator_pyramid.append(Discriminator(input_dim=3, dim=dim, n_layers=5))

In [6]:
noise_optimal_pyramid = []
for scale in range(config['num_scale']):
    if not scale:
        noise = torch.randn_like(real_pyramid[scale])
    else:
        noise = torch.zeros_like(real_pyramid[scale])
    noise_optimal_pyramid.append(noise)

In [7]:
for scale in range(config['num_scale']):
    print(real_pyramid[scale].shape, noise_optimal_pyramid[scale].shape)

torch.Size([1, 3, 25, 37]) torch.Size([1, 3, 25, 37])
torch.Size([1, 3, 33, 49]) torch.Size([1, 3, 33, 49])
torch.Size([1, 3, 43, 64]) torch.Size([1, 3, 43, 64])
torch.Size([1, 3, 56, 83]) torch.Size([1, 3, 56, 83])
torch.Size([1, 3, 73, 109]) torch.Size([1, 3, 73, 109])
torch.Size([1, 3, 96, 143]) torch.Size([1, 3, 96, 143])
torch.Size([1, 3, 125, 187]) torch.Size([1, 3, 125, 187])


In [35]:
tuple(real_pyramid[0].shape)

(1, 3, 25, 37)

In [34]:
torch.ones(torch.ones(tu`ple(real_pyramid[0].shape)))

TypeError: ones(): argument 'size' (position 1) must be tuple of ints, not Tensor

### dev

In [6]:
scale = 0

In [19]:
generator = generator_pyramid[scale]
discriminator = generator_pyramid[scale]

real = real_pyramid[0]
real= torch.tensor(np.transpose(real, [2, 0, 1])[np.newaxis])

In [20]:
real.shape
generator(real).shape

torch.Size([1, 3, 25, 37])

torch.Size([1, 3, 11, 23])

In [24]:
real.shape
real[:, :, 5:-5, 5:-5].shape

torch.Size([1, 3, 25, 37])

torch.Size([1, 3, 15, 27])